# Temporal Visualization & Drift Analysis

This notebook demonstrates temporal drift visualization capabilities for:
- Bin-level temporal drift analysis
- Distribution shift detection (PSI)
- Score stability monitoring
- Multi-snapshot comparisons
- Segmentation support

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from cr_score.viz import BinningVisualizer, ScoreVisualizer

pd.set_option('display.max_columns', None)
print("Imports successful")

## 1. Create Sample Temporal Data

In [ ]:
# Create sample credit bureau data across multiple snapshots
np.random.seed(42)

snapshots = ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06']
bins = ['[0-30]', '[31-50]', '[51-100]', '[101-150]']
n_customers = 1000

data = []
for snapshot in snapshots:
    # Introduce drift over time - event rates increase
    drift_factor = 1 + (snapshots.index(snapshot) * 0.1)
    
    for customer_id in range(n_customers):
        bin_val = np.random.choice(bins)
        
        # Base event rate by bin
        base_rate = 0.1 if bin_val == '[0-30]' else (0.2 if bin_val == '[31-50]' else 0.3)
        event_rate = min(0.9, base_rate * drift_factor)
        
        data.append({
            'customer_id': f'CUST_{customer_id:04d}',
            'snapshot': snapshot,
            'age_bin': bin_val,
            'age': np.random.randint(18, 80),
            'target': np.random.choice([0, 1], p=[1-event_rate, event_rate]),
            'credit_score': np.random.randint(300, 850),
            'segment': np.random.choice(['credit_card', 'personal_loan']),
        })

df = pd.DataFrame(data)
print(f"Created dataset: {len(df)} rows")
print(f"Snapshots: {df['snapshot'].unique()}")
print(f"\nSample data:")
df.head(10)

## 2. Bin-Level Temporal Drift Visualization

In [ ]:
viz = BinningVisualizer()

# Plot temporal drift with confidence bands
fig = viz.plot_temporal_bin_drift(
    df,
    feature_col='age_bin',
    target_col='target',
    snapshot_col='snapshot',
    snapshot_values=['2024-01', '2024-03', '2024-06'],
    baseline_snapshot='2024-01',
    show_confidence_bands=True,
    max_bins_display=10,
)

fig.show()

## 3. Delta vs Baseline Analysis

In [ ]:
# Plot delta (change) vs baseline
fig = viz.plot_bin_delta_vs_baseline(
    df,
    feature_col='age_bin',
    target_col='target',
    snapshot_col='snapshot',
    baseline_snapshot='2024-01',
    comparison_snapshots=['2024-03', '2024-06'],
)

fig.show()

## 4. PSI Visualization (Population Stability Index)

In [ ]:
# Plot PSI over time
fig = viz.plot_psi_by_feature(
    df,
    feature_col='age',
    snapshot_col='snapshot',
    baseline_snapshot='2024-01',
    n_bins=10,
)

fig.show()

print("PSI Thresholds:")
print("  < 0.10: Low drift (acceptable)")
print("  0.10-0.25: Medium drift (monitor closely)")
print("  > 0.25: High drift (action required)")

## 5. Score-Level Temporal Stability

In [ ]:
score_viz = ScoreVisualizer()

# Score distribution over time
fig = score_viz.plot_temporal_score_distribution(
    df,
    score_col='credit_score',
    snapshot_col='snapshot',
    target_col='target',
    snapshot_values=['2024-01', '2024-03', '2024-06'],
)

fig.show()

In [ ]:
# KS curve comparison across snapshots
fig = score_viz.plot_temporal_ks_comparison(
    df,
    score_col='credit_score',
    target_col='target',
    snapshot_col='snapshot',
    snapshot_values=['2024-01', '2024-03', '2024-06'],
)

fig.show()

In [ ]:
# Stability metrics dashboard
fig = score_viz.plot_temporal_stability_metrics(
    df,
    score_col='credit_score',
    target_col='target',
    snapshot_col='snapshot',
    approval_threshold=600,
)

fig.show()

## 6. Segmentation Support

In [ ]:
# Plot temporal drift by segment
fig = viz.plot_temporal_bin_drift(
    df,
    feature_col='age_bin',
    target_col='target',
    snapshot_col='snapshot',
    segment_col='segment',
    segment_values=['credit_card'],
    baseline_snapshot='2024-01',
)

fig.show()

## 7. Export with Metadata

In [ ]:
# Create visualization
fig = viz.plot_temporal_bin_drift(
    df,
    feature_col='age_bin',
    target_col='target',
    snapshot_col='snapshot',
    baseline_snapshot='2024-01',
)

# Export with metadata
metadata = {
    'feature_name': 'age_bin',
    'model_id': 'v2.1',
    'snapshot_range': '2024-01 to 2024-06',
    'baseline_snapshot': '2024-01',
    'segment': 'all',
}

# Note: Uncomment to export
# viz._export_figure_with_metadata(
#     fig,
#     path='reports/temporal_drift_age_bin.html',
#     format='html',
#     metadata=metadata
# )

print("Figure ready for export with metadata")

## 8. Complete Temporal Drift Analysis Workflow

In [ ]:
# Comprehensive analysis workflow
print("Temporal Drift Analysis Workflow:")
print("=" * 80)

# 1. Check PSI for distribution shift
print("\n1. PSI Analysis (Distribution Shift):")
fig_psi = viz.plot_psi_by_feature(df, 'age', 'snapshot', '2024-01')
fig_psi.show()

# 2. Check bin-level drift
print("\n2. Bin-Level Drift Analysis:")
fig_drift = viz.plot_temporal_bin_drift(
    df, 'age_bin', 'target', 'snapshot', baseline_snapshot='2024-01'
)
fig_drift.show()

# 3. Check delta vs baseline
print("\n3. Delta vs Baseline:")
fig_delta = viz.plot_bin_delta_vs_baseline(
    df, 'age_bin', 'target', 'snapshot', baseline_snapshot='2024-01'
)
fig_delta.show()

# 4. Check score stability
print("\n4. Score Stability Metrics:")
fig_stability = score_viz.plot_temporal_stability_metrics(
    df, 'credit_score', 'target', 'snapshot', approval_threshold=600
)
fig_stability.show()

print("\n" + "=" * 80)
print("Complete! All temporal drift analyses generated.")

## Summary

This notebook demonstrated:

1. ✅ **Bin-Level Temporal Drift**: Event rate and population % across snapshots
2. ✅ **Delta vs Baseline**: Change detection per bin
3. ✅ **PSI Visualization**: Distribution shift detection
4. ✅ **Score Stability**: Distribution, KS, and stability metrics
5. ✅ **Segmentation**: Filtered analysis by segment
6. ✅ **Export with Metadata**: Audit-ready visualizations

All temporal visualization methods are production-ready and backward compatible with existing code.